In [1]:
import pandas as pd
import numpy as np
import wikipedia

In [2]:
wikipedia.__version__

(1, 4, 0)

In [3]:
data_cols = ['user id','movie id','rating','timestamp']
data = pd.read_csv('../data/ml-100k/u.data', sep='\t', header=None, names=data_cols)
data.head()

,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
item_cols = ['movie id',  'movie title', 'release date', 'video release date','IMDb URL', 'unknown',\
'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama','Fantasy', \
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance','Sci-Fi', 'Thriller', 'War', 'Western']
item = pd.read_csv('../data/ml-100k/u.item',  sep='|',header=None,names=item_cols,encoding='latin-1')
item.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
# Assuming 'df' is your DataFrame
genre_columns = ['Action', 'Adventure', 'Animation', 'Children', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# Function to get genres for a row
def get_genres(row):
    genres = [genre for genre in genre_columns if row[genre] == 1]
    return ', '.join(genres) if genres else 'unknown'

# Apply the function to each row
item['Genres'] = item.apply(get_genres, axis=1)
item.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Genres
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,"Animation, Children"
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,"Action, Adventure, Thriller"
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Action
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller


In [6]:
item = item[['movie id', 'movie title', 'Genres']]
item.head()

,movie id,movie title,Genres
0,1,Toy Story (1995),"Animation, Children"
1,2,GoldenEye (1995),"Action, Adventure, Thriller"
2,3,Four Rooms (1995),Thriller
3,4,Get Shorty (1995),Action
4,5,Copycat (1995),Thriller


In [7]:
# merge data and item
df = pd.merge(data, item, on='movie id').sort_values(by=['user id', 'movie id']).reset_index(drop=True)
df.head()

,user id,movie id,rating,timestamp,movie title,Genres
0,1,1,5,874965758,Toy Story (1995),"Animation, Children"
1,1,2,3,876893171,GoldenEye (1995),"Action, Adventure, Thriller"
2,1,3,4,878542960,Four Rooms (1995),Thriller
3,1,4,3,876893119,Get Shorty (1995),Action
4,1,5,3,889751712,Copycat (1995),Thriller


In [8]:
# average rating for each movie for each user, add the column to df
df['avg_rating'] = df.groupby(['user id', 'movie id'])['rating'].transform('mean')
# rename the column
df.columns = ['user_id', 'movie_id', 'rating', 'timestamp', 'movie_title', 'genres', 'avg_rating']
df.head()

,user_id,movie_id,rating,timestamp,movie_title,genres,avg_rating
0,1,1,5,874965758,Toy Story (1995),"Animation, Children",5.0
1,1,2,3,876893171,GoldenEye (1995),"Action, Adventure, Thriller",3.0
2,1,3,4,878542960,Four Rooms (1995),Thriller,4.0
3,1,4,3,876893119,Get Shorty (1995),Action,3.0
4,1,5,3,889751712,Copycat (1995),Thriller,3.0


In [9]:
df.to_csv('../data/processed_movie100k.csv', index=False)

### Add wiki summary

In [10]:
movie100k = pd.read_csv('D:/test/LLM-Recommender-System-with-RAG/data/processed_movie100k.csv')
movie100k.head()
print(movie100k.shape)

(100000, 7)


In [11]:
movie_data = []

for title in np.unique(movie100k['movie_title']):
    try:
        # Formatting the title for Wikipedia
        title_formatted = title.replace("'", "").rstrip()
        if "(" in title_formatted and ")" in title_formatted:
            title_formatted = title_formatted.replace(")", " film)")  # Add 'film' before the closing parenthesis
        else:
            title_formatted += " (film)"

        # Fetching the summary
        summary = wikipedia.summary(title_formatted, sentences=1)
        movie_data.append({'movie_title': title, 'wiki_summary': summary})
    except wikipedia.DisambiguationError as e:
        # Handle disambiguation by looking for a title containing 'film'
        relevant_page = next((option for option in e.options if "film" in option and title.split(" (")[0] in option), None)
        if relevant_page:
            try:
                summary = wikipedia.summary(relevant_page, sentences=1)
                movie_data.append({'movie_title': title, 'wiki_summary': summary})
            except Exception as ex:
                print(f"Failed to fetch summary for {title}: {ex}")
        else:
            print(f"No relevant Wikipedia page found for {title}")
    except Exception as ex:
        print(f"Failed to fetch summary for {title}: {ex}")

# Creating the DataFrame after collecting all data
movie_wiki = pd.DataFrame(movie_data)

Failed to fetch summary for 187 (1997): Page id "187 1994 film" does not match any pages. Try another id!
Failed to fetch summary for 2001: A Space Odyssey (1968): Page id "2001 a space odyssey 1998 film" does not match any pages. Try another id!
Failed to fetch summary for Absolute Power (1997): Page id "absolute power 1999 film" does not match any pages. Try another id!
Failed to fetch summary for Alice in Wonderland (1951): Page id "alice in wonderland 1981 film" does not match any pages. Try another id!
Failed to fetch summary for Amityville 3-D (1983): Page id "amityville 3 d 1993 film" does not match any pages. Try another id!
Failed to fetch summary for April Fool's Day (1986): Page id "april fool's day 1986 film" does not match any pages. Try another id!
Failed to fetch summary for Audrey Rose (1977): Page id "audrey rose 1979 film" does not match any pages. Try another id!
Failed to fetch summary for Babe (1995): Page id "babe 1999 film" does not match any pages. Try another i

c:\Users\Admin\.conda\envs\env1\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\Admin\.conda\envs\env1\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Failed to fetch summary for Beyond Bedlam (1993): Page id "beyond bedlam 1997 film" does not match any pages. Try another id!
No relevant Wikipedia page found for Big Bang Theory, The (1994)
Failed to fetch summary for Bitter Sugar (Azucar Amargo) (1996): Page id "bitter sugar azúcar amarga film 1996 film" does not match any pages. Try another id!
Failed to fetch summary for Blink (1994): Page id "blank 1998 film" does not match any pages. Try another id!
Failed to fetch summary for Blood & Wine (1997): Page id "bloom wind 1997 film" does not match any pages. Try another id!
Failed to fetch summary for Bloody Child, The (1996): Page id "blonde child the 1996 film" does not match any pages. Try another id!
Failed to fetch summary for Bram Stoker's Dracula (1992): Page id "bram stoker's dracula 1992 film" does not match any pages. Try another id!
Failed to fetch summary for Brothers in Trouble (1995): Page id "brothers in trouble 1996 film" does not match any pages. Try another id!
Faile

KeyboardInterrupt: 

In [ ]:
movie_wiki.head()

In [ ]:
movie_wiki.to_csv('D:/test/LLM-Recommender-System-with-RAG/data/movie_wiki.csv', index=False)